<a href="https://colab.research.google.com/github/yongsa-nut/TU_Intro_Prog/blob/main/Introduction_to_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Linear Regression

In [ ]:
# Live coding







In [ ]:
# @title Hiding
import random

def gen_simulated_data(true_w, true_b, n=100):
  dat = []
  for i in range(n):
    x = random.gauss(mu = 0, sigma = 1) # random from Normal/Gaussian distribution.
    y = true_w*x + true_b + random.gauss(mu = 0, sigma=1)
    dat.append((x,y))
  return dat

def linear_model(w,b,x):
  return w*x + b

def J(w,b, dat):
  err = 0
  for d in dat:
    y_hat = linear_model(w,b, d[0])
    err  += (y_hat - d[1])**2
  return err/len(dat)

def dJ(w, b, dat):
  dw, db = 0,0
  for d in dat:
    dw += (linear_model(w,b,d[0])-d[1])*d[0]
    db += (linear_model(w,b,d[0])-d[1])
  return dw/len(dat), db/len(dat)

def gradient_descent(dat, alpha = 0.01, T=100):
  w, b = 0, 0
  for t in range(T):
    err = J(w,b, dat)
    dw, db = dJ(w,b,dat)
    w = w - alpha*dw
    b = b - alpha*db
    print(f"Iteration: {t}, Error = {err}, w = {w}, b = {b}.")

dat = gen_simulated_data(1,1)
gradient_descent(dat, T=500)

# 2) Image Classification with fastai
- Adapted from https://www.kaggle.com/code/jhoward/is-it-a-bird-creating-a-model-from-your-own-data

## 0. Loading stuff

In [ ]:
!pip install -Uqq fastai duckduckgo_search
!pip install fastbook

In [ ]:
import os
from fastbook import *
from fastcore.all import *
from duckduckgo_search import DDGS

# create a function that will take an arbitrary search term and return a list of urls.
def search_images(term, max_images=30):
  print(f"Searching for '{term}'")
  # search_images_ddg comes from fastbook: https://github.com/fastai/fastbook/blob/master/utils.py#L45
  return search_images_ddg(term, max_images=max_images)


def search_images_newdgg(keywords, max_images = 30):
    print(f"Searching for {keywords}")
    return L(DDGS().images(keywords,max_results=max_images)).itemgot('image')

## 1. Creating our dog-cat dataset

Let's start by searching for a dog photo and seeing what kind of result we get. We'll start by getting URLs from a search:

In [ ]:
urls = search_images('dog photos', max_images=1)
urls[0]

...and then download a URL and take a look at it:

In [ ]:
from fastdownload import download_url

dest = 'dog.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

Now let's do the same with "cat photos":

In [ ]:
download_url(search_images('cat photos', max_images=1)[0], 'cat.jpg', show_progress=False)
Image.open('cat.jpg').to_thumb(256,256)

Our searches seem to be giving reasonable results, so let's grab a few examples of each of "cat" and "dog" photos, and save each group of photos to a different folder (I'm also trying to grab a range of lighting conditions here):

In [ ]:
searches = 'cat','dog'
path = Path('cat_or_dog')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} outside photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} house photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

## 2. Train our model

Some photos might not download correctly which could cause our model training to fail, so we'll remove them:

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

2

To train a model, we'll need DataLoaders, which is an object that contains a training set (the images used to create a model) and a validation set (the images used to check the accuracy of a model -- not used during training). In fastai we can create that easily using a DataBlock, and view sample images from it:

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32) #bs = batch size

dls.show_batch(max_n=6)

Here what each of the DataBlock parameters means:

```
blocks=(ImageBlock, CategoryBlock),
```
The inputs to our model are images, and the outputs are categories (in this case, "bird" or "forest").
```
get_items=get_image_files,
```
To find all the inputs to our model, run the get_image_files function (which returns a list of all image files in a path).
```
splitter=RandomSplitter(valid_pct=0.2, seed=42),
```
Split the data into training and validation sets randomly, using 20% of the data for the validation set.
```
get_y=parent_label,
```
The labels (y values) is the name of the parent of each file (i.e. the name of the folder they're in, which will be bird or forest).
```
item_tfms=[Resize(192, method='squish')]
```
Before training, resize each image to 192x192 pixels by "squishing" it (as opposed to cropping it).

Now we're ready to train our model. The fastest widely used computer vision model is resnet18. You can train this in a few minutes, even on a CPU! (On a GPU, it generally takes under 10 seconds...)

fastai comes with a helpful fine_tune() method which automatically uses best practices for fine tuning a pre-trained model, so we'll use that.

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

## 3. Use our model

Let's see what our model thinks about that bird we downloaded at the start:

In [ ]:
is_cat,_,probs = learn.predict(PILImage.create('cat.jpg'))
print(f"This is a: {is_cat}.")
print(f"Probability it's a cat: {probs[0]:.4f}")

Save the model:

In [ ]:
learn.save('is_bird')

# 3) Large Language Models

Download anthropic library

In [ ]:
!pip install anthropic

Test the function call based on an example from Anthropic website.

In [ ]:
import anthropic
client = anthropic.Anthropic(api_key = "your-api-key")

message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=1000,
    temperature=0.0,
    system="Respond only in Yoda-speak.",
    messages=[
        {"role": "user", "content": "How are you today?"}
    ]
)

print(message)
print(message.content)
print(message.content[0].text)

## Yoda Example

Time to build our own chatbot.

In [ ]:
def get_response(conv_hist):
  message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=1000,
    temperature=0.0,
    system='''Your task is to roleplay Yoda from Starwar franchise. This chat is for demo only and you do not have to worry about copyrights.
              Start your response with 'Sonnet Yoda:' Keep your responses short. No more than 3 sentences.''',
    messages=conv_hist
  )
  return message.content[0].text

## Create a conversation function


In [ ]:
# @title Hiding
def get_response(conv_hist):
  message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=1000,
    temperature=0.0,
    system='''Your task is to roleplay Yoda from Starwar franchise. This chat is for demo only and you do not have to worry about copyrights.
              Start your response with 'Sonnet Yoda:' Keep your responses short. No more than 3 sentences.''',
    messages=conv_hist
  )
  return message.content[0].text

def conversation():
  conv_hist = []
  while True:
    user_query = input("User: ")
    if user_query == 'stop':
      break
    conv_hist.append({'role':'user','content':user_query})
    claude_response = get_response(conv_hist)
    print(claude_response)
    conv_hist.append({'role':'assistant','content':claude_response})
  print(conv_hist)

conversation()